In [10]:
import plotly.express as px
import pandas as pd

points = [[12.53019946, 55.7111808], [12.53018721, 55.71117492], [12.53017498, 55.71116905], [12.53016273, 55.71116317], [12.53015047, 55.7111573]]
df = pd.DataFrame(points, columns=['long', 'lat'])
# Create a DataFrame from the points list
df = pd.DataFrame(points, columns=['long', 'lat'])

# Add an 'id' column
df['id'] = range(1, len(df) + 1)

# Reorder columns to match the format in the picture
df = df[['id', 'lat', 'long']]

# Display the DataFrame
print(df)

fig = px.scatter_geo(df,lat='lat',lon='long', hover_name="id")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

   id        lat       long
0   1  55.711181  12.530199
1   2  55.711175  12.530187
2   3  55.711169  12.530175
3   4  55.711163  12.530163
4   5  55.711157  12.530150


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [11]:
import plotly.express as px
px.set_mapbox_access_token(open(".mapbox_token").read())
df = px.data.carshare()
fig = px.scatter_mapbox(df, lat="centroid_lat", lon="centroid_lon",     color="peak_hour", size="car_hours",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=10)
fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '.mapbox_token'

In [15]:
L = ['0', '1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '23', '24', '3', '4', '5', '6', '7', '8', '9']

# Convert list items to integers
int_list = [int(x) for x in L]

# sort the list, remove the two smallest and two largest numbers and Convert list items back to strings
result = [str(x) for x in sorted(int_list)[2:-2]]

print(result)


['2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22']
